Here are listed all the modules, libraries, imports and constant used for the exercise

In [264]:
from EnumReviews import enumReviews
import string


FILE_INPUT = "reviews.txt"

This function is used to read the file as a list of list of words (strings) and label (integer) 

In [265]:
def load_file(path2file:str)->list[list[str, int]]:
    data = []
    
    with open(path2file, "r", encoding="UTF-8") as fp:
        fp.readline()
        
        for line in fp:
            line=line.strip().split(",")
            data.append([",".join(line[:-1]), int(line[-1])])
          
    return data

This function applies the tokenization 

In [266]:
def tokenize(data:list[list[str, int]])->list[list[str]]:
    """Compute the tokens for each document.
    Input: a list of strings. Each item is a document to tokenize.
    Output: a list of lists. Each item is a list containing the tokens of the
    relative document.
    """
    enum = enumReviews()
    tokens = []
    for doc in data:
        doc = doc[enum.DESC]
        for punct in string.punctuation:
            doc = doc.replace(punct, " ")
        split_doc = [token.lower() for token in doc.split(" ") if token]
        tokens.append(split_doc)
    return tokens


This function counts the number of occurrence of words in a list of tokens

In [267]:
def countFrequencies(token:list[list[str]])->list[dict[str:int]]:
    # sourcery skip: inline-immediately-returned-variable, move-assign-in-block
    # No good it's cubic !
    # return [{word:token[i].count(word) for word in token[i]} for i in range(len(token))]

    # This is quadratic, which is better
    data = []
    for text in token:
        diz = dict()
        for word in text:
            if word in diz:
                diz[word]+=1
            else:
                diz[word]=1
        data.append(diz)
    
    return data
    

This function computes the df of each word in a list of list of tokens

In [268]:
def computeDF(freq:list[dict[str:int]], words:set[str])->dict[str:int]:
    pass
    
    

This function computes the IDF for each word in the text and returns the list in ascending order

In [269]:
def computeIDF(freq:list[dict[str:int]])->list[list[str, float]]:
    df = computeDF(freq, {key for doc in freq for key in doc.keys()})
    

This is the main function of the program

In [270]:
def main()->None:
    data = load_file(FILE_INPUT)
    token = tokenize(data)
    frequencies = countFrequencies(token)
    idf = computeIDF(frequencies)
    
main()